In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt

# Cosas a solucionar/decisiones
- Hay que quitar del entrenamiento las palabras que llevan espacio, CRISTHIANO RONALDO y MINERIA DE DATOS, dan problemas al sacar la feature de numero de errores



In [7]:
data = pd.read_csv('train.csv', header=None ,delimiter=";" )
feature_names = ['usuario', 'palabra', 'palabraLeida', 'tiempoCaracter',
       'hayErrPalabra', 'tiempoErrPalabra', 'numPalabra','tiempoPalabra', 'tamPalabra', 'caracter',
       'falloCaracter', 'palabraCorrecta']
data.columns = feature_names

In [8]:
data.head()


,usuario,palabra,palabraLeida,tiempoCaracter,hayErrPalabra,tiempoErrPalabra,numPalabra,tiempoPalabra,tamPalabra,caracter,falloCaracter,palabraCorrecta
0,Cristhian,ZANAHORIA,Z,0.348,False,0.0,0,0.0,9,Z,False,False
1,Cristhian,ZANAHORIA,ZA,0.490,False,0.0,0,0.0,9,A,False,False
2,Cristhian,ZANAHORIA,ZAN,0.702,False,0.0,0,0.0,9,N,False,False
3,Cristhian,ZANAHORIA,ZANA,0.807,False,0.0,0,0.0,9,A,False,False
4,Cristhian,ZANAHORIA,ZANAH,0.273,False,0.0,0,0.0,9,H,False,False


# Mapear los valores verdadero y falso a 1 y 0
hayErrPalabra, falloCaracter, palabraCorrecta

In [9]:
# Pasamos de boolean a un int, 1 para true y 0 para false
data["hayErrPalabra"] = data['hayErrPalabra'].map({False: 0, True: 1})
data["falloCaracter"] = data['falloCaracter'].map({False: 0, True: 1})
data["palabraCorrecta"] = data['palabraCorrecta'].map({False: 0, True: 1})

# Quitarle los espacios en blanco al usuario

In [10]:
data["usuario"] = data["usuario"].str.strip()

# Mapear el usuario en un campo usuarioID

In [11]:
data["usuarioID"] = data['usuario'].map({"Cristhian": 0, "Jesus": 1})

# (Mirar si interesa hacer o no)
# Mapear cada palabra a un numero para poder entrenar
Primero se crea un diccionar almacenando cada valor unico y luego se recorre cambiado los valores

In [334]:
d = {ni: indi for indi, ni in enumerate(set(data['palabra']))}
data['palabra'] = [d[ni] for ni in data['palabra']]

# (Mirar si interesa hacer o no)
# Mapear cada caracter a un numero para poder entrenar
Primero se crea un diccionar almacenando cada valor unico y luego se recorre cambiado los valores

In [335]:
d = {ni: indi for indi, ni in enumerate(set(data['caracter']))}
data['caracter'] = [d[ni] for ni in data['caracter']]

# Sacar tiempo medio de escritura del mismo caracter
Hay que quitar los caracteres nulos

In [31]:
caracter = data[~data['caracter'].isnull()][['usuario', 'caracter','tiempoCaracter','falloCaracter']]
caracter['user'] = data['usuarioID']
caracter = caracter.groupby(['usuario','caracter']).mean()
targerCaracter = caracter['user']
caracter = caracter.drop(['user'], axis=1)
caracter.iloc[0:3]

tiempoCaracter  falloCaracter
usuario   caracter                               
Cristhian                 0.338200       1.000000
          A               0.338054       0.196429
          C               0.077744       0.046512

# Sacar tiempo medio de pulsado de enter (caracteres nulos)

In [342]:
Enter = data[data['caracter'].isnull()][['usuario','tiempoCaracter']]
Enter.columns = ['usuario', 'tiempoEnter']
Enter = Enter.groupby(['usuario']).mean()
Enter

,tiempoEnter
usuario,
Cristhian,0.363036
Jesus,0.499133


# Usuario, palabra, tiempo

In [343]:
usPalTiempo = data[data['caracter'].isnull()][['usuario', 'palabra', 'tiempoPalabra', 'tiempoErrPalabra','tamPalabra']]
usPalTiempo

,usuario,palabra,tiempoPalabra,tiempoErrPalabra,tamPalabra
9,Cristhian,ZANAHORIA,7.342,0.000,9
20,Cristhian,PANTALON,3.773,0.000,8
33,Cristhian,PQINZLVNREUS,8.024,0.000,12
52,Cristhian,MINERIA DE DATOS,0.000,0.000,16
72,Cristhian,MINERIA DE DATOS,12.541,6.871,16
91,Cristhian,CRISTHIANO RONALDO,5.548,0.000,18
104,Cristhian,MIMIMIMIMIMI,4.488,0.000,12
111,Cristhian,QWMNZP,5.207,0.000,6
126,Cristhian,PZKOFTLILILILI,7.343,0.000,14
137,Cristhian,PELIAGUDO,4.585,0.000,9


# Sacar la suma de fallos totales por palabra

In [370]:
falloCaracterPorPalabra = data.groupby(['usuario','palabra'])['falloCaracter'].sum()
falloCaracterPorPalabra

usuario    palabra           
Cristhian  COMPOSICION            3
           COMPRENSION            7
           CORRECCION             0
           CRISTHIANO            38
           CRISTHIANO RONALDO     0
           DIMENSION              0
           MIMIMIMIMIMI           0
           MINERIA               57
           MINERIA DE DATOS       0
           PANTALON               2
           PASION                 0
           PELIAGUDO              3
           PQINZLVNREUS           0
           PRECISION              1
           PRESION                1
           PRIVACION              2
           PZKOFTLILILILI         0
           QWMNZP                 5
           TENSION                0
           TRADICION              3
           ZANAHORIA              2
           ZIZIZIHIZIZHIHIHI      1
Jesus      COMPOSICION            3
           COMPRENSION            1
           CORRECCION             1
           CRISTHIANO            27
           CRISTHIANO RONALDO     

In [366]:
tiempoCoreccionCaracter = data[data['falloCaracter'] > 0].groupby(['usuario','palabra'])['tiempoCaracter'].sum()
tiempoCoreccionCaracter

usuario    palabra          
Cristhian  COMPOSICION           1.041
           COMPRENSION           2.437
           CRISTHIANO           14.162
           MINERIA              23.449
           PANTALON              0.759
           PELIAGUDO             1.061
           PRECISION             0.197
           PRESION               0.271
           PRIVACION             0.686
           QWMNZP                3.318
           TRADICION            -0.130
           ZANAHORIA             0.845
           ZIZIZIHIZIZHIHIHI     0.314
Jesus      COMPOSICION           1.679
           COMPRENSION           0.453
           CORRECCION            0.507
           CRISTHIANO           13.505
           DIMENSION             1.662
           MINERIA              12.226
           PASION                1.831
           PQINZLVNREUS          0.594
           PRECISION             0.504
           PZKOFTLILILILI        1.536
           QWMNZP                1.175
           TENSION               0.

# Error en el entreno, hay un tiempo negativo, MIRAR

In [372]:
dataFallo = data[data['tiempoErrPalabra'] > 0]
dataFallo[dataFallo['palabra'] == "PZKOFTLILILILI"]

,usuario,palabra,palabraLeida,tiempoCaracter,hayErrPalabra,tiempoErrPalabra,numPalabra,tiempoPalabra,tamPalabra,caracter,falloCaracter,palabraCorrecta,usuarioID
1349,Cristhian,PZKOFTLILILILI,PZKOFTLILILILI,0.319,0,7.324,8,13.564,14,NaN,0,1,0


# Sacar el tiempoPalabra medio de cada palabra del usuario para usarlo como modelo
TiempoErrPalabra no se si es muy util

In [345]:
tiempoMedioPalabra = usPalTiempo.drop(['tamPalabra'], axis=1)
tiempoMedioPalabra['user'] = data['usuarioID']
#usPalTiempo2['numPalabra'] = usPalTiempo['palabra']
tiempoMedioPalabra = tiempoMedioPalabra.groupby(['usuario','palabra']).mean()
tiempoMedioPalabra['falloCaracterPorPalabra'] = falloCaracterPorPalabra
targetTM = tiempoMedioPalabra['user']
tiempoMedioPalabra = tiempoMedioPalabra.drop(['user'], axis=1)
tiempoMedioPalabra

tiempoPalabra  tiempoErrPalabra  \
usuario   palabra                                               
Cristhian COMPOSICION              4.325250            0.0000   
          COMPRENSION              0.000000            0.0000   
          CORRECCION               3.596000            0.0000   
          CRISTHIANO RONALDO       5.775250            0.0000   
          DIMENSION                3.669500            0.0000   
          MIMIMIMIMIMI             4.500600            1.0824   
          MINERIA DE DATOS         5.634800            1.3742   
          PANTALON                 3.632500            0.0000   
          PASION                   2.704250            0.0000   
          PELIAGUDO                4.403000            0.0000   
          PQINZLVNREUS             6.851500            0.0000   
          PRECISION                3.544750            0.0000   
          PRESION                  3.312250            0.0000   
          PRIVACION                4.236000            0.0000   
          PZKOFTLILILILI           7.012600            1.4648   
          QWMNZP                   5.225600            0.8652   
          TENSION                  2.808500            0.0000   
          TRADICION                3.533500            0.0000   
          ZANAHORIA                6.888000            0.0000   
          ZIZIZIHIZIZHIHIHI        6.641500            0.0000   
Jesus     COMPOSICION              4.637000            0.0000   
          COMPRENSION              0.000000            0.0000   
          CORRECCION               3.851667            0.0000   
          CRISTHIANO RONALDO       6.752333            0.0000   
          DIMENSION                4.034000            0.0000   
          MIMIMIMIMIMI             4.628333            0.0000   
          MINERIA DE DATOS         4.306000            0.0000   
          PANTALON                 3.067000            0.0000   
          PASION                   2.843667            0.0000   
          PELIAGUDO                4.388333            0.0000   
          PQINZLVNREUS             6.816333            0.0000   
          PRECISION                3.649667            0.0000   
          PRESION                  2.748333            0.0000   
          PRIVACION                3.554000            0.0000   
          PZKOFTLILILILI           7.969333            0.0000   
          QWMNZP                   4.603000            0.0000   
          TENSION                  2.860000            0.0000   
          TRADICION                4.923667            0.0000   
          ZANAHORIA                7.215000            0.0000   
          ZIZIZIHIZIZHIHIHI        7.899000            0.0000   

                              falloCaracterPorPalabra  
usuario   palabra                                      
Cristhian COMPOSICION                               3  
          COMPRENSION                               7  
          CORRECCION                                0  
          CRISTHIANO RONALDO                        0  
          DIMENSION                                 0  
          MIMIMIMIMIMI                              0  
          MINERIA DE DATOS                          0  
          PANTALON                                  2  
          PASION                                    0  
          PELIAGUDO                                 3  
          PQINZLVNREUS                              0  
          PRECISION                                 1  
          PRESION                                   1  
          PRIVACION                                 2  
          PZKOFTLILILILI                            0  
          QWMNZP                                    5  
          TENSION                                   0  
          TRADICION                                 3  
          ZANAHORIA                                 2  
          ZIZIZIHIZIZHIHIHI                         1  
Jesus     COMPOSICION                               3  
          C

# Sacar tiempo medio por caracter por tamaño de palabra

In [226]:
usPalTiempo3 = usPalTiempo.drop(['palabra'], axis=1)
targetUS = usPalTiempo3['usuario']
usPalTiempo3 = usPalTiempo3.groupby(['usuario']).mean()
#usPalTiempo3['tiempoMedioCaracter'] = usPalTiempo3['tiempoPalabra'] / usPalTiempo3['tamPalabra']
usPalTiempo3

,tiempoPalabra,tiempoErrPalabra,tamPalabra
usuario,,,
Cristhian,4.470893,0.284917,10.52381
Jesus,4.537333,0.000000,10.45000


In [67]:
usPalTiempo3['tiempoEnter'] = Enter
usPalTiempo3


,tiempoPalabra,tamPalabra,tiempoEnter
usuario,,,
0,4.470893,10.52381,0.363036
1,4.537333,10.45000,0.499133


In [227]:
data

,usuario,palabra,palabraLeida,tiempoCaracter,hayErrPalabra,tiempoErrPalabra,numPalabra,tiempoPalabra,tamPalabra,caracter,falloCaracter,palabraCorrecta,usuarioID
0,Cristhian,3,Z,0.348,0,0.0,0,0.000,9,25,0,0,0
1,Cristhian,3,ZA,0.490,0,0.0,0,0.000,9,3,0,0,0
2,Cristhian,3,ZAN,0.702,0,0.0,0,0.000,9,15,0,0,0
3,Cristhian,3,ZANA,0.807,0,0.0,0,0.000,9,3,0,0,0
4,Cristhian,3,ZANAH,0.273,0,0.0,0,0.000,9,10,0,0,0
5,Cristhian,3,ZANAHO,0.276,0,0.0,0,0.000,9,14,0,0,0
6,Cristhian,3,ZANAHOR,0.352,0,0.0,0,0.000,9,19,0,0,0
7,Cristhian,3,ZANAHORI,0.321,0,0.0,0,0.000,9,9,0,0,0
8,Cristhian,3,ZANAHORIA,0.281,0,0.0,0,0.000,9,3,0,0,0
9,Cristhian,3,ZANAHORIA,0.369,0,0.0,1,7.342,9,0,0,0,0


# Sacar el target

In [87]:
target = data['usuarioID']
target

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
1687    1
1688    1
1689    1
1690    1
1691    1
1692    1
1693    1
1694    1
1695    1
1696    1
1697    1
1698    1
1699    1
1700    1
1701    1
1702    1
1703    1
1704    1
1705    1
1706    1
1707    1
1708    1
1709    1
1710    1
1711    1
1712    1
1713    1
1714    1
1715    1
1716    1
Name: usuarioID, dtype: int64

# Eliminar campos sobrantes (Usuario, palabra, palabraLeida, numPalabra, tamPalabra, caracter, usuarioID)

In [303]:
data = data.drop(['usuario','palabraLeida','numPalabra', 'tamPalabra','usuarioID'], axis=1)

In [304]:
#'palabra', (mirar estos) 'falloCaracter' 'palabraCorrecta', 'hayErrPalabra'
data

,palabra,tiempoCaracter,hayErrPalabra,tiempoErrPalabra,tiempoPalabra,caracter,falloCaracter,palabraCorrecta
0,3,0.348,0,0.0,0.000,25,0,0
1,3,0.490,0,0.0,0.000,3,0,0
2,3,0.702,0,0.0,0.000,15,0,0
3,3,0.807,0,0.0,0.000,3,0,0
4,3,0.273,0,0.0,0.000,10,0,0
5,3,0.276,0,0.0,0.000,14,0,0
6,3,0.352,0,0.0,0.000,19,0,0
7,3,0.321,0,0.0,0.000,9,0,0
8,3,0.281,0,0.0,0.000,3,0,0
9,3,0.369,0,0.0,7.342,0,0,1


# Cambiar datos malos por las mejoras

In [211]:
tiempoPorPalabra = data[data['tiempoErrPalabra'] > 0][['palabra','tiempoPalabra', 'tiempoErrPalabra', 'palabraCorrecta']]

tiempoPorPalabra


,palabra,tiempoPalabra,tiempoErrPalabra,palabraCorrecta
72,9,12.541,6.871,0
360,10,11.486,4.326,0
1314,5,10.362,5.412,0
1349,14,13.564,7.324,0


In [161]:
#data['tiempoPalabra'] = [tiempoPorPalabra['tiempoPalabra'] for tiempoPorPalabra['tiempoPalabra'] in data['tiempoPalabra']]
data2 = data.copy()

In [163]:
data2 = data2.drop(['tiempoPalabra', 'tiempoErrPalabra'], axis=1)


In [144]:
#data2["tiempoPalabra"] = data2["palabra"].map(tiempoPorPalabra)

In [165]:
data2

,palabra,tiempoCaracter,caracter
0,3,0.348,25
1,3,0.490,3
2,3,0.702,15
3,3,0.807,3
4,3,0.273,10
5,3,0.276,14
6,3,0.352,19
7,3,0.321,9
8,3,0.281,3
9,3,0.369,0


In [164]:
data

,palabra,tiempoCaracter,tiempoErrPalabra,tiempoPalabra,caracter
0,3,0.348,0.0,0.000,25
1,3,0.490,0.0,0.000,3
2,3,0.702,0.0,0.000,15
3,3,0.807,0.0,0.000,3
4,3,0.273,0.0,0.000,10
5,3,0.276,0.0,0.000,14
6,3,0.352,0.0,0.000,19
7,3,0.321,0.0,0.000,9
8,3,0.281,0.0,0.000,3
9,3,0.369,0.0,7.342,0


# Separar datos de entreno y datos de testeo

# Cross Validation


# Random Forest

In [305]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=101)
scores = cross_val_score(random_forest, data, target, cv=5)
print(scores)
print(scores.mean())

[ 0.61627907  0.6627907   0.62973761  0.68804665  0.66472303]
0.652315411214


# SVM

In [306]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

svm = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(svm, data, target, cv=5)
print(scores)
print(scores.mean())

[ 0.66569767  0.61918605  0.63556851  0.67638484  0.67055394]
0.653478201912


# AdaBoost

datos originales

In [16]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

ada = AdaBoostClassifier(n_estimators=100)

scores = cross_val_score(ada, data, target, cv=5)
print(scores)
print(scores.mean())

NameError: name 'target' is not defined

# Pruebas con otro modelo

In [301]:
scores = cross_val_score(ada, tiempoMedioPalabra, targetTM, cv=5)
print(scores)
print(scores.mean())

[ 0.625  0.75   0.625  0.625  0.5  ]
0.625


# Pruebas con modelo tiempo medio por caracter

In [44]:
scores = cross_val_score(ada, caracter, targerCaracter, cv=5)
print(scores)
print(scores.mean())

[ 0.9         0.7         1.          0.9         0.88888889]
0.877777777778


# Necesito Entranar el modelo caracter, con los datos sin el target

In [45]:
# no se si estaria bien asi ya que caracter tiene el usuario
ada.fit(caracter,targerCaracter)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)

# Necesito Predecir con una nueva fila
por ejemplo
[caracter, tiempo_caracter, tiempo_error]

seria algo asi 

modelo.predict([[1,0.23,0.02]])

In [47]:

ada.predict([[0.23,0.22]])

array([ 0.])

In [43]:
caracter

tiempoCaracter  falloCaracter
usuario   caracter                               
Cristhian                 0.338200       1.000000
          A               0.338054       0.196429
          C               0.077744       0.046512
          D               0.302692       0.538462
          E               0.243455       0.181818
          F               0.682000       0.000000
          G               0.403000       0.333333
          H               0.478500       0.076923
          I               0.414669       0.034286
          K               1.084200       0.000000
          L               0.422457       0.114286
          M               0.344481       0.019231
          N               0.438958       0.126316
          O               0.400629       0.216495
          P               0.103917       0.041667
          Q               0.249333       0.000000
          R               0.133440       0.120000
          S               0.430957       0.255319
          T               0.170630       0.222222
          U               0.435250       0.000000
          V               0.918875       0.000000
          W               0.420200       0.000000
          Z               0.345289       0.000000
          d               0.234000       1.000000
Jesus                     0.487222       1.000000
          <               0.869500       1.000000
          A               0.612025       0.175000
          C               0.579094       0.062500
          D               0.567895       0.526316
          E               0.687355       0.129032
          F               1.305750       0.250000
          G               0.867000       0.000000
          H               0.764056       0.000000
          I               0.634130       0.024390
          K               0.825333       0.000000
          L               0.643417       0.125000
          M               0.642000       0.029412
          N               0.615882       0.073529
          O               0.578211       0.197183
          P               0.730771       0.057143
          Q               0.711333       0.000000
          R               0.666306       0.083333
          S               0.588774       0.129032
          T               0.763053       0.210526
          U               0.764167       0.000000
          V               0.793333       0.000000
          W               0.530000       0.000000
          X               1.845000       1.000000
          Z               0.701643       0.035714